In [1]:
from collections import Counter
import numpy as np
from math import ceil
import cv2

<div class="alert alert-block alert-info">
    <b>Zigzag: </b>Amacı matris elemanlarını düşük frekanstan yüksek frekansa dogru artacak biçimde dizmektir.
Böylece ilk sıradaki eleman DC katsayı olacaktır. Kuantalama blogunda sıfırlanan elemanlar ise 
burada yan yana gelecektir.Bu durum kodlama kısmında kolaylık saglayacaktır.
</div>

In [2]:
def zigzag(matrix: np.ndarray) -> np.ndarray:
    # nicelenmiş bir bloğun zigzagını hesaplar.
    # değişkenleri başlatma
    h = 0
    v = 0
    v_min = 0
    h_min = 0
    v_max = matrix.shape[0]
    h_max = matrix.shape[1]
    i = 0
    output = np.zeros((v_max * h_max))

    while (v < v_max) and (h < h_max):
        if ((h + v) % 2) == 0:  # yukarı çık
            if v == v_min:
                output[i] = matrix[v, h]  # İlk satır
                if h == h_max:
                    v = v + 1
                else:
                    h = h + 1
                i = i + 1
            elif (h == h_max - 1) and (v < v_max):  # son sütun
                output[i] = matrix[v, h]
                v = v + 1
                i = i + 1
            elif (v > v_min) and (h < h_max - 1):  # diğer tüm durumlar
                output[i] = matrix[v, h]
                v = v - 1
                h = h + 1
                i = i + 1
        else:  # aşağı git
            if (v == v_max - 1) and (h <= h_max - 1):  # son satır
                output[i] = matrix[v, h]
                h = h + 1
                i = i + 1
            elif h == h_min:  # ilk sütun
                output[i] = matrix[v, h]
                if v == v_max - 1:
                    h = h + 1
                else:
                    v = v + 1
                i = i + 1
            elif (v < v_max - 1) and (h > h_min):  # diğer tüm durumlar
                output[i] = matrix[v, h]
                v = v + 1
                h = h - 1
                i = i + 1
        if (v == v_max - 1) and (h == h_max - 1):  # sağ alt eleman
            output[i] = matrix[v, h]
            break
    return output

In [3]:
def trim(array: np.ndarray) -> np.ndarray:
    """
    trim_zeros(): diziden baştaki ve/veya sondaki sıfırları kırpın.
    trim_zeros işlevinin boş bir dizi döndürmesi durumunda, DC bileşeni olarak kullanmak için diziye bir sıfır ekleyin
    """
    trimmed = np.trim_zeros(array, 'b') # arkadaki sıfırları siler
    if len(trimmed) == 0:
        trimmed = np.zeros(1)
    return trimmed

<div class="alert alert-block alert-info">
<b>AC Dizi Uzunlugu (Run Length) Kodlama</b><br>
Kuantalama ve zig-zag bloklarından sonra içinde birçok sıfır içeren dizi elde edilmişti.
Bu aşamada her 8x8’lik blokta yer alan 63 adet AC katsayı kodlanır. 
Örnek olarak 57, 45, 0, 0, 0, 0, 23, 0, -30, -16, 0, 0, 1, 0 .... 0 şeklinde bir dizi varsa 
AC dizi uzunlugu kodlaması şu şekilde olacaktır: (0,57); (0,45); (4,23); (1,-30); (0,-16); (2,1); EOB.
EOB (End of Block) kodu özel olarak belirlenmiştir. Eger zig-zag blogundan gelen vektörün kalan 
elemanlarının hepsi 0 ise EOB blogu konularak bütün sıfırların gönderilmesi engellenebilir. 
Eger zig-zag blogundan gelen vektör 0 ile bitmiyorsa, yani son eleman 0 degil ise, EOB koyulmaz.
</div>

In [4]:
def run_length_encoding(array: np.ndarray) -> list:    # çalışma uzunluğu kodlaması
    """
    zikzakları temsil eden ara akışı bulur
    DC bileşenlerinin formatı <size><amplitude (genişlik)> şeklindedir.
    AC bileşenleri için biçim <run_length, size> <Amplitude of non-zero> şeklindedir.
    :param numpy.ndarray array: dizideki zikzak vektörleri
    """
    encoded = list()
    run_length = 0
    eob = ("EOB",)

    for i in range(len(array)):
        for j in range(len(array[i])):
            trimmed = trim(array[i]) # arkadaki sıfırlar silindi.
            if j == len(trimmed):
                encoded.append(eob)  # Arkadaki sıfırların olduğu konuma geldiğinde EOB ekle.
                break
            if i == 0 and j == 0:  # ilk DC bileşeni için
                encoded.append((int(trimmed[j]).bit_length(), trimmed[j]))
# int.bit_length(): İşaret ve baştaki sıfırlar hariç, bir tamsayıyı ikili sistemde temsil etmek için gereken bit sayısını döndürür.
            elif j == 0:  # DC bileşenleri arasındaki farkı hesaplamak için
                diff = int(array[i][j] - array[i - 1][j])
                if diff != 0:
                    encoded.append((diff.bit_length(), diff))
                else:
                    encoded.append((1, diff))
                run_length = 0
            elif trimmed[j] == 0:  # sıfır olması durumunda çalışma_uzunluğunu bir artırın
                run_length += 1
            else:  # AC bileşenlerinin ara buhar gösterimi
                encoded.append((run_length, int(trimmed[j]).bit_length(), trimmed[j]))
                run_length = 0
            # EOB gönder
        if not (encoded[len(encoded) - 1] == eob):
            encoded.append(eob)
    return encoded

In [5]:
def get_freq_dict(array: list) -> dict:
    """
    tuşların dizinin değerleri olduğu ve değerlerin frekansları olduğu bir sözlük döndürür.
    :param numpy.ndarray array: dizi olarak ara akış
    :return: frekans tablosu
    """
    data = Counter(array) # Nesneleri anahtar ve değer olarak sayan bir sözlüktür.
    result = {k: d / len(array) for k, d in data.items()}
    return result

In [6]:
def find_huffman(p: dict) -> dict:

    # Yalnızca iki simgenin temel durumu, keyfi olarak 0 veya 1 atayın; frekans önemli değil
    if len(p) == 2:
        return dict(zip(p.keys(), ['0', '1']))

    # Olası en düşük çifti birleştirerek yeni bir dağıtım oluşturun
    p_prime = p.copy()
    a1, a2 = lowest_prob_pair(p)
    p1, p2 = p_prime.pop(a1), p_prime.pop(a2)
    p_prime[a1 + a2] = p1 + p2

    # Yeni dağıtımda kodu yineleyin ve oluşturun
    c = find_huffman(p_prime)
    ca1a2 = c.pop(a1 + a2)
    c[a1], c[a2] = ca1a2 + '0', ca1a2 + '1'

    return c

In [7]:
def lowest_prob_pair(p):
    # En düşük olasılıkla p dağılımından sembol çiftini döndür
    sorted_p = sorted(p.items(), key=lambda x: x[1])
    return sorted_p[0][0], sorted_p[1][0]

In [8]:
# Niceleme tablolarını tanımlayalım.
QTY = np.array([[16, 11, 10, 16, 24, 40, 51, 61],  # luminance quantization table - Parlaklık
                [12, 12, 14, 19, 26, 48, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]])

QTC = np.array([[17, 18, 24, 47, 99, 99, 99, 99],  # chrominance quantization table - Renklilik
                [18, 21, 26, 66, 99, 99, 99, 99],
                [24, 26, 56, 99, 99, 99, 99, 99],
                [47, 66, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99]])
# define window size
windowSize = len(QTY)  # 8

In [9]:
# read image
imgOriginal = cv2.imread('marbles.bmp', cv2.IMREAD_COLOR)

# convert BGR to YCrCb
img = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2YCR_CB)
width = len(img[0]) # 1419 sütun
height = len(img)   # 1001 satır
print("image: ", img.shape)

image:  (1001, 1419, 3)


In [10]:
y = np.zeros((height, width), np.float32) + img[:, :, 0]
cr = np.zeros((height, width), np.float32) + img[:, :, 1]
cb = np.zeros((height, width), np.float32) + img[:, :, 2]

# görüntünün sıkıştırılmadan önceki bit cinsinden boyutu  -> satır * sütun * 8
totalNumberOfBitsWithoutCompression = len(y) * len(y[0]) * 8 + len(cb) * len(cb[0]) * 8 + len(cr) * len(cr[0]) * 8
print("Görüntünün sıkıştırılmadan önceki bit cinsinden boyutu: ", totalNumberOfBitsWithoutCompression)

Görüntünün sıkıştırılmadan önceki bit cinsinden boyutu:  34090056


In [11]:
# kanal değerleri normalleştirilmelidir, bu nedenle 128 çıkarıyoruz.
y = y - 128
cr = cr - 128
cb = cb - 128

In [12]:
# 4: 2: 2 alt örnekleme kullanılır. (başka bir alt örnekleme şeması da kullanılabilir)
# bu nedenle krominans (renk) kanalları alt örneklenmelidir
# alt örnekleme faktörlerini hem yatay hem de dikey yönlerde tanımlıyoruz.
SSH, SSV = 2, 2

# 2x2 ortalama filtre kullanarak krominans kanallarını filtreleyelim. (başka bir filtre türü kullanılabilir)
# boxFilter() - ortalama alma bulanıklaştırma işlemine benzer; bir filtreye ikili bir görüntü uygular.
# ddepth - Çıktı görüntüsünün derinliğini temsil eden tamsayı türünde bir değişken.
# ksize - Bulanıklaştıran çekirdeğin boyutunu temsil eden bir Size nesnesi.
crf = cv2.boxFilter(cr, ddepth=-1, ksize=(2, 2))
cbf = cv2.boxFilter(cb, ddepth=-1, ksize=(2, 2))
crSub = crf[::SSV, ::SSH]
cbSub = cbf[::SSV, ::SSH]

print(len(crf))
print(len(crSub))

1001
501


In [13]:
# dolgu gerekip gerekmediğini kontrol ediyoruz,
# Eğer dolgu gerekli ise her kanalın DCT'sini sıfırlarla doldurmak için boş diziler tanımlayalım.
yWidth, yLength = ceil(len(y[0]) / windowSize) * windowSize, ceil(len(y) / windowSize) * windowSize
# print("y_width: ", ceil(len(y[0]) / windowSize) * windowSize)  # 1419 / 8 = 117.375 = 178 * 8 = 1424
# print("y_length", ceil(len(y) / windowSize) * windowSize)       # 1001 / 8 = 125.125 = 126 * 8 = 1008
if (len(y[0]) % windowSize == 0) and (len(y) % windowSize == 0):  # 1419 % 8 = 3 and 1001 % 8 = 1  (sıfırlarla doldurmamız gerek)
    yPadded = y.copy()
else:
    yPadded = np.zeros((yLength, yWidth))  # (1008, 1424) boyutlu sıfırlardan oluşan dizinin
    for i in range(len(y)):  # 1001
        for j in range(len(y[0])):  # 1419
            yPadded[i, j] += y[i, j]       # ilgili konumlarına eski verileri ekliyoruz. y[0,0]=(-128.0)=yPadded[0,0] gibi

# renklilik (chrominance) kanalları aynı boyutlara sahiptir, yani her ikisi de bir döngüde doldurulabilir
cWidth, cLength = ceil(len(cbSub[0]) / windowSize) * windowSize, ceil(len(cbSub) / windowSize) * windowSize # 712, 504
if (len(cbSub[0]) % windowSize == 0) and (len(cbSub) % windowSize == 0): # 710 % 8 = 6 and 501 % 8 = 5 (sıfırlarla doldurmamız gerek)
    crPadded = crSub.copy()
    cbPadded = cbSub.copy()
else:
    crPadded = np.zeros((cLength, cWidth)) # (504, 712)
    cbPadded = np.zeros((cLength, cWidth))
    for i in range(len(crSub)): # 501
        for j in range(len(crSub[0])): # 710
            crPadded[i, j] += crSub[i, j]
            cbPadded[i, j] += cbSub[i, j]

In [14]:
# her kanalın DCT'sini al
# üç boş matris tanımla
yDct, crDct, cbDct = np.zeros((yLength, yWidth)), np.zeros((cLength, cWidth)), np.zeros((cLength, cWidth))

# parlaklık kosinüs dönüşüm değerlerini hesaplamak için x ekseni ve y ekseni üzerindeki yineleme sayısı
hBlocksForY = int(len(yDct[0]) / windowSize)  # parlaklık için yatay yönde blok sayısı 178
vBlocksForY = int(len(yDct) / windowSize)  # parlaklık için dikey yönde blok sayısı 126
# krominans kanallarını hesaplamak için x ekseni ve y eksenindeki yineleme sayısı kosinüs dönüşümleri değerleri
hBlocksForC = int(len(crDct[0]) / windowSize)  # renklilik için yatay yönde blok sayısı 89
vBlocksForC = int(len(crDct) / windowSize)  # renklilik için dikey yönde blok sayısı 63

# nicelenmiş değerleri depolamak için 3 boş matris tanımlayın
yq, crq, cbq = np.zeros((yLength, yWidth)), np.zeros((cLength, cWidth)), np.zeros((cLength, cWidth))
# ve zikzaklar için 3 tane daha
yZigzag = np.zeros(((vBlocksForY * hBlocksForY), windowSize * windowSize))
crZigzag = np.zeros(((vBlocksForC * hBlocksForC), windowSize * windowSize))
cbZigzag = np.zeros(((vBlocksForC * hBlocksForC), windowSize * windowSize))

In [15]:
yCounter = 0
for i in range(vBlocksForY): # 126
    for j in range(hBlocksForY): # 178
        # Ayrık Kosinüs Dönüşümü
        yDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = cv2.dct(
            yPadded[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        # Niceleme
        yq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = np.ceil(
            yDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] / QTY)
        # Zigzag
        yZigzag[yCounter] += zigzag(
            yq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        yCounter += 1
yZigzag = yZigzag.astype(np.int16)

# blok sayısını hesaplamak için crq veya cbq kullanılabilir.
cCounter = 0
for i in range(vBlocksForC):
    for j in range(hBlocksForC):
        crDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = cv2.dct(
            crPadded[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        crq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = np.ceil(
            crDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] / QTC)
        crZigzag[cCounter] += zigzag(
            crq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        cbDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = cv2.dct(
            cbPadded[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        cbq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] = np.ceil(
            cbDct[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize] / QTC)
        cbZigzag[cCounter] += zigzag(
            cbq[i * windowSize: i * windowSize + windowSize, j * windowSize: j * windowSize + windowSize])
        cCounter += 1
crZigzag = crZigzag.astype(np.int16)
cbZigzag = cbZigzag.astype(np.int16)

In [16]:
# her kanal için çalışma uzunluğu kodlamasını bulun
# daha sonra bir Huffman sözlüğü oluşturmak için her bileşenin frekansını alın
yEncoded = run_length_encoding(yZigzag)
yFrequencyTable = get_freq_dict(yEncoded)
yHuffman = find_huffman(yFrequencyTable)

crEncoded = run_length_encoding(crZigzag)
crFrequencyTable = get_freq_dict(crEncoded)
crHuffman = find_huffman(crFrequencyTable)

cbEncoded = run_length_encoding(cbZigzag)
cbFrequencyTable = get_freq_dict(cbEncoded)
cbHuffman = find_huffman(cbFrequencyTable)

In [17]:
# Her kanal için iletilecek bit sayısını hesaplayın
# ve bunları bir çıktı dosyasına yazın
file = open("CompressedImage.asfh", "w")
yBitsToTransmit = str()
for value in yEncoded:
    yBitsToTransmit += yHuffman[value]

crBitsToTransmit = str()
for value in crEncoded:
    crBitsToTransmit += crHuffman[value]

cbBitsToTransmit = str()
for value in cbEncoded:
    cbBitsToTransmit += cbHuffman[value]

if file.writable():
    file.write(yBitsToTransmit + "\n" + crBitsToTransmit + "\n" + cbBitsToTransmit)
file.close()

# Sıkıştırmadan Sonra Toplam Bit Sayısı
totalNumberOfBitsAfterCompression = len(yBitsToTransmit) + len(crBitsToTransmit) + len(cbBitsToTransmit)
# 2198128 + 457967 + 465325 = 3121420

print(
    "Sıkıştırma Oranı: " + str(
        np.round(totalNumberOfBitsWithoutCompression / totalNumberOfBitsAfterCompression, 1)))

# Sıkıştırma Olmadan Toplam Bit Sayısı / Sıkıştırmadan Sonra Toplam Bit Sayısı = 34090056 / 3121420 = 10.9

Sıkıştırma Oranı: 10.9
